## 0. Create Spark session

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext
import pyspark.sql.functions as F

# Variables d'environnement
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--packages org.apache.hadoop:hadoop-aws:3.2.0,org.apache.hadoop:hadoop-common:3.2.0,io.trino:trino-jdbc:422 '
    'pyspark-shell'
)
os.environ['S3_ENDPOINT'] = "http://minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"

# On reprend la config S3 de l'autre fichier
spark = (
    SparkSession.builder
    .appName("spark-silver-to-gold")
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID"))
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY"))
    .config("spark.hadoop.fs.s3a.endpoint", os.getenv("S3_ENDPOINT"))
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.attempts.maximum", "1")
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "5000")
    .config("spark.hadoop.fs.s3a.connection.timeout", "10000")
    .getOrCreate()
)


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
io.trino#trino-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-84286c1b-d6c5-4124-b9e6-678b7845d4ae;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found org.apache.hadoop#hadoop-common;3.2.0 in central
	found org.apache.hadoop#hadoop-annotations;3.2.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found org.apache.httpcomponents#httpclient;4.5.2 in central
	found org.apache.httpcomponents#httpcore;4.4.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	f

## 1. Extract from object storage

In [2]:
spark.sparkContext.setLogLevel("WARN")

#lire la table en parquet comme on lisait le json
silver_path = "s3a://velib/silver/velib-disponibilite-en-temps-reel"
df_silver = spark.read.parquet(silver_path)

# Verifications
print("Schema:")
df_silver.printSchema()
print("\nSample data:")
df_silver.show(5, truncate=False)
print("\nTotal records:", df_silver.count())

24/10/31 14:10:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Schema:
root
 |-- capacity: long (nullable = true)
 |-- duedate: string (nullable = true)
 |-- ebike: long (nullable = true)
 |-- is_installed: string (nullable = true)
 |-- is_renting: string (nullable = true)
 |-- is_returning: string (nullable = true)
 |-- mechanical: long (nullable = true)
 |-- name: string (nullable = true)
 |-- nom_arrondissement_communes: string (nullable = true)
 |-- numbikesavailable: long (nullable = true)
 |-- numdocksavailable: long (nullable = true)
 |-- stationcode: string (nullable = true)
 |-- fill_ratio: double (nullable = true)
 |-- part_minute: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- part_day: date (nullable = true)


Sample data:


+--------+-------------------------+-----+------------+----------+------------+----------+-----------------------------+---------------------------+-----------------+-----------------+-----------+--------------------+----------------+-----------------+------------------+----------+
|capacity|duedate                  |ebike|is_installed|is_renting|is_returning|mechanical|name                         |nom_arrondissement_communes|numbikesavailable|numdocksavailable|stationcode|fill_ratio          |part_minute     |lat              |lon               |part_day  |
+--------+-------------------------+-----+------------+----------+------------+----------+-----------------------------+---------------------------+-----------------+-----------------+-----------+--------------------+----------------+-----------------+------------------+----------+
|35      |2024-10-29T10:05:10+00:00|0    |OUI         |OUI       |OUI         |0         |Benjamin Godard - Victor Hugo|Paris                      |0  


Total records: 1088469


# Cleaning of df_silver

In [3]:
df_silver = df_silver.filter(F.col("part_minute") >= F.lit("2024-10-07T17:16"))
print("\nTotal records:", df_silver.count())
df_silver.show()


Total records: 1061237
+--------+--------------------+-----+------------+----------+------------+----------+--------------------+---------------------------+-----------------+-----------------+-----------+--------------------+----------------+------------------+------------------+----------+
|capacity|             duedate|ebike|is_installed|is_renting|is_returning|mechanical|                name|nom_arrondissement_communes|numbikesavailable|numdocksavailable|stationcode|          fill_ratio|     part_minute|               lat|               lon|  part_day|
+--------+--------------------+-----+------------+----------+------------+----------+--------------------+---------------------------+-----------------+-----------------+-----------+--------------------+----------------+------------------+------------------+----------+
|      35|2024-10-29T10:05:...|    0|         OUI|       OUI|         OUI|         0|Benjamin Godard -...|                      Paris|                0|              

## 2. Turnover Rate by Station

In [17]:
from pyspark.sql.window import Window

# Define the window partitioned by station and ordered by time
window_spec = Window.partitionBy("stationcode").orderBy("duedate")

# Calculate the turnover rate (number of bikes - number of bikes before)
df_turnover = df_silver \
    .withColumn("prev_numbikesavailable", F.lag("numbikesavailable").over(window_spec)) \
    .withColumn("turnover_rate", F.abs(F.col("numbikesavailable") - F.col("prev_numbikesavailable"))) \
    .groupBy("stationcode", "part_day", "part_minute") \
    .agg(F.sum("turnover_rate").alias("total_turnover"))

df_turnover.show()

+-----------+----------+----------------+--------------+
|stationcode|  part_day|     part_minute|total_turnover|
+-----------+----------+----------------+--------------+
|      26005|2024-10-29|2024-10-29T10:08|            29|
|      26005|2024-10-29|2024-10-29T11:05|            22|
|      26005|2024-10-29|2024-10-29T12:11|            46|
|      26005|2024-10-29|2024-10-29T13:06|            49|
|      26005|2024-10-29|2024-10-29T14:11|            23|
|      26005|2024-10-29|2024-10-29T15:11|             8|
|      26005|2024-10-30|2024-10-30T09:11|            49|
|      26005|2024-10-30|2024-10-30T10:11|            58|
|      26005|2024-10-30|2024-10-30T11:12|            33|
|      26005|2024-10-30|2024-10-30T12:12|            32|
|      26005|2024-10-30|2024-10-30T13:11|            57|
|      26005|2024-10-30|2024-10-30T14:11|            87|
|      26005|2024-10-30|2024-10-30T15:11|            24|
|      26005|2024-10-31|2024-10-31T07:09|            25|
|      26005|2024-10-31|2024-10

## 3. Basic Statistics on a range of time

In [16]:
# Aggregate statistics for a range of time
df_stats = df_silver \
    .groupBy("part_day", "part_minute") \
    .agg(
        F.avg("numbikesavailable").alias("avg_bikes"),
        F.expr("percentile(numbikesavailable, 0.5)").alias("median_bikes"),
        F.min("numbikesavailable").alias("min_bikes"),
        F.max("numbikesavailable").alias("max_bikes"),
        F.avg("fill_ratio").alias("avg_fill_ratio")
    ).orderBy("part_minute")

df_stats.show()

+----------+----------------+------------------+------------+---------+---------+-------------------+
|  part_day|     part_minute|         avg_bikes|median_bikes|min_bikes|max_bikes|     avg_fill_ratio|
+----------+----------------+------------------+------------+---------+---------+-------------------+
|2024-10-28|2024-10-28T11:07|              31.0|        31.0|       31|       31|                1.0|
|2024-10-28|2024-10-28T19:53|             16.15|        16.0|       16|       18|0.46118326118326075|
|2024-10-29|2024-10-29T10:01| 3.565811965811966|         2.0|        0|       13|0.15596863421097143|
|2024-10-29|2024-10-29T10:02| 6.766222222222222|         4.0|        0|       31|0.22523548431172086|
|2024-10-29|2024-10-29T10:03|3.3045751633986926|         1.0|        0|       25|0.11747876205492498|
|2024-10-29|2024-10-29T10:04| 5.073703703703703|         2.0|        0|       46|0.17597481756807037|
|2024-10-29|2024-10-29T10:05| 4.852525252525252|         3.0|        0|       45| 

## 4. Basic Statistics per station

In [6]:
# Aggregate statistics per station
df_stats_stations = df_silver \
    .groupBy("stationcode","part_day", "part_minute", "lat", "lon") \
    .agg(
        F.avg("numbikesavailable").alias("avg_bikes"),
        F.expr("percentile(numbikesavailable, 0.5)").alias("median_bikes"),
        F.min("numbikesavailable").alias("min_bikes"),
        F.max("numbikesavailable").alias("max_bikes"),
        F.avg("fill_ratio").alias("avg_fill_ratio")
    ).orderBy("part_minute")

df_stats_stations.toPandas().head()

,stationcode,part_day,part_minute,lat,lon,avg_bikes,median_bikes,min_bikes,max_bikes,avg_fill_ratio
0,8018,2024-10-28,2024-10-28T11:07,48.876201,2.319786,31.000000,31.0,31,31,1.000000
1,46002,2024-10-28,2024-10-28T19:53,48.764615,2.407119,16.141414,16.0,16,18,0.461183
2,None,2024-10-28,2024-10-28T19:53,NaN,NaN,17.000000,17.0,17,17,NaN
3,23206,2024-10-29,2024-10-29T10:01,48.804048,2.284826,1.066667,1.0,1,2,0.038095
4,9114,2024-10-29,2024-10-29T10:01,48.880883,2.349934,0.000000,0.0,0,0,0.000000


In [13]:
df_stats_stations.filter(F.col("stationcode") == "46002").show()
df_stats_stations.describe()

+-----------+----------+----------------+------------------+------------------+------------------+------------+---------+---------+-------------------+
|stationcode|  part_day|     part_minute|               lat|               lon|         avg_bikes|median_bikes|min_bikes|max_bikes|     avg_fill_ratio|
+-----------+----------+----------------+------------------+------------------+------------------+------------+---------+---------+-------------------+
|      46002|2024-10-28|2024-10-28T19:53|48.764615483395254|2.4071189761161804|16.141414141414142|        16.0|       16|       18|0.46118326118326075|
|      46002|2024-10-29|2024-10-29T12:09|48.764615483395254|2.4071189761161804|24.892857142857142|        25.0|       19|       32| 0.7112244897959189|
|      46002|2024-10-29|2024-10-29T13:11|48.764615483395254|2.4071189761161804| 29.92156862745098|        30.0|       29|       30| 0.8549019607843132|
|      46002|2024-10-29|2024-10-29T14:11|48.764615483395254|2.4071189761161804|29.385964

DataFrame[summary: string, stationcode: string, part_minute: string, lat: string, lon: string, avg_bikes: string, median_bikes: string, min_bikes: string, max_bikes: string, avg_fill_ratio: string]

## 5. Save aggregates in minio

In [8]:
# Define S3 path for gold tables
os.environ['s3_output_path_gold_turnover'] = "s3a://velib/gold/turnover_rate/"
os.environ['s3_output_path_gold_stats'] = "s3a://velib/gold/stats/"
os.environ['s3_output_path_gold_stats_stations'] = "s3a://velib/gold/stats_stations/"

spark.sparkContext.setLogLevel("WARN")

# Write the turnover rate DataFrame
df_turnover.write \
    .partitionBy("stationcode") \
    .format("parquet") \
    .mode("overwrite") \
    .save(os.getenv('s3_output_path_gold_turnover'))

# Write the stats DataFrame
df_stats.write \
    .partitionBy("part_day") \
    .format("parquet") \
    .mode("overwrite") \
    .save(os.getenv('s3_output_path_gold_stats'))

df_stats_stations.write \
    .partitionBy("part_day") \
    .format("parquet") \
    .mode("overwrite") \
    .save(os.getenv('s3_output_path_gold_stats_stations'))


24/10/31 14:10:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/10/31 14:10:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/10/31 14:10:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/10/31 14:10:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/31 14:10:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/10/31 14:10:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/10/31 14:10:53 WARN MemoryManager: Total allocation exceeds 95.

## 6. Saving tables into trino

In [15]:
import trino

host, port, user = 'trino-coordinator', 8080, 'trino'
conn = trino.dbapi.connect(host=host, port=port, user=user)
cur = conn.cursor()

catalog, schema, table_turnover, table_stats, table_stats_stations = 'minio', 'velib_gold', 'turnover_rate', 'stats', 'stats_stations'
schema_turnover, schema_stats ,schema_stats_stations = 'velib_gold_turnover', 'velib_gold_stats', 'velib_gold_stats_stations'
schema_location = 's3a://velib/gold/'

turnover_partitioned_by, stats_partitioned_by, stats_stations_partitioned_by= 'stationcode', 'part_day', 'part_day'

turnover_external_location = 's3a://velib/gold/turnover_rate/'
stats_external_location = 's3a://velib/gold/stats/'
stats_stations_external_location = 's3a://velib/gold/stats_stations/'


# Now define the queries, using the paths defined above
queries = [
    f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_turnover} WITH (location = '{schema_location}')",
    f"DROP TABLE IF EXISTS {catalog}.{schema_turnover}.{table_turnover}",
    f"""
    CREATE TABLE IF NOT EXISTS {catalog}.{schema_turnover}.{table_turnover} (
        part_day DATE,
        part_minute VARCHAR,
        total_turnover BIGINT,
        stationcode BIGINT
    ) 
    WITH (
        format = 'PARQUET',
        partitioned_by = ARRAY['{turnover_partitioned_by}'],
        external_location = '{turnover_external_location}'
    )
    """,
    f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_stats} WITH (location = '{schema_location}')",
    f"DROP TABLE IF EXISTS {catalog}.{schema_stats}.{table_stats}",
    f"""
    CREATE TABLE IF NOT EXISTS {catalog}.{schema_stats}.{table_stats} (
        part_minute VARCHAR,
        avg_bikes DOUBLE,
        median_bikes DOUBLE,
        min_bikes BIGINT,
        max_bikes BIGINT,
        avg_fill_ratio DOUBLE,
        part_day DATE
    ) 
    WITH (
        format = 'PARQUET',
        partitioned_by = ARRAY['{stats_partitioned_by}'],
        external_location = '{stats_external_location}'
    )
    """,
    f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_stats_stations} WITH (location = '{schema_location}')",
    f"DROP TABLE IF EXISTS {catalog}.{schema_stats_stations}.{table_stats_stations}",
    f"""
    CREATE TABLE IF NOT EXISTS {catalog}.{schema_stats_stations}.{table_stats_stations} (
        stationcode VARCHAR,
        part_minute VARCHAR,
        lat DOUBLE,
        lon DOUBLE,
        avg_bikes DOUBLE,
        median_bikes DOUBLE,
        min_bikes BIGINT,
        max_bikes BIGINT,
        avg_fill_ratio DOUBLE,
        part_day DATE
    ) 
    WITH (
        format = 'PARQUET',
        partitioned_by = ARRAY['{stats_stations_partitioned_by}'],
        external_location = '{stats_stations_external_location}'
    )
    """,
    f"USE {catalog}.{schema_turnover}",
    f"CALL system.sync_partition_metadata('{schema_turnover}', 'turnover_rate', 'ADD')",
    f"USE {catalog}.{schema_stats}",
    f"CALL system.sync_partition_metadata('{schema_stats}', 'stats', 'ADD')",
    f"USE {catalog}.{schema_stats_stations}",
    f"CALL system.sync_partition_metadata('{schema_stats_stations}', '{table_stats_stations}', 'ADD')"
]

# Execute each query in the list
for query in queries:
    try:
        cur.execute(query)
        # Check if the query is a SELECT query to fetch results
        if query.startswith("SELECT"):
            results = cur.fetchall()
            for row in results:
                print(row)
        else:
            print(f"Executed: {query}")
    except Exception as e:
        print(f"Error executing query: {query}. Error: {e}")

# Close the cursor and connection
cur.close()
conn.close()

Executed: CREATE SCHEMA IF NOT EXISTS minio.velib_gold_turnover WITH (location = 's3a://velib/gold/')
Executed: DROP TABLE IF EXISTS minio.velib_gold_turnover.turnover_rate
Executed: 
    CREATE TABLE IF NOT EXISTS minio.velib_gold_turnover.turnover_rate (
        part_day DATE,
        part_minute VARCHAR,
        total_turnover BIGINT,
        stationcode BIGINT
    ) 
    WITH (
        format = 'PARQUET',
        partitioned_by = ARRAY['stationcode'],
        external_location = 's3a://velib/gold/turnover_rate/'
    )
    
Executed: CREATE SCHEMA IF NOT EXISTS minio.velib_gold_stats WITH (location = 's3a://velib/gold/')
Executed: DROP TABLE IF EXISTS minio.velib_gold_stats.stats
Executed: 
    CREATE TABLE IF NOT EXISTS minio.velib_gold_stats.stats (
        part_minute VARCHAR,
        avg_bikes DOUBLE,
        median_bikes DOUBLE,
        min_bikes BIGINT,
        max_bikes BIGINT,
        avg_fill_ratio DOUBLE,
        part_day DATE
    ) 
    WITH (
        format = 'PARQUET',
